In [1]:
import numpy as np
import pandas as pd
from urbansim.utils import misc

In [2]:
store = pd.HDFStore('model_data.h5')
blocks = store['blocks']
jobs = store['jobs']
households = store['households']
residential_units = store['residential_units']

In [3]:
def rent(residential_units, blocks):
    blocks = blocks[['res_rents', 'tract_id', 'puma10_id']]

    tract_vals = pd.DataFrame(index=np.unique(blocks.tract_id), data=blocks[blocks.res_rents > 0].groupby('tract_id').res_rents.median()).fillna(0)
    puma_vals = pd.DataFrame(index=np.unique(blocks.puma10_id), data=blocks[blocks.res_rents > 0].groupby('puma10_id').res_rents.median()).fillna(0)

    blocks['tract_vals'] = misc.reindex(tract_vals.res_rents, blocks.tract_id)
    blocks['puma_vals'] = misc.reindex(puma_vals.res_rents, blocks.puma10_id)

    blocks.res_rents[blocks.res_rents == 0] = blocks.tract_vals[blocks.res_rents == 0]
    blocks.res_rents[blocks.res_rents == 0] = blocks.puma_vals[blocks.res_rents == 0]

    return misc.reindex(blocks.res_rents, residential_units.block_id)
    
def value(residential_units, blocks):
    blocks = blocks[['res_values', 'tract_id', 'puma10_id']]

    tract_vals = pd.DataFrame(index=np.unique(blocks.tract_id), data=blocks[blocks.res_values > 0].groupby('tract_id').res_values.median()).fillna(0)
    puma_vals = pd.DataFrame(index=np.unique(blocks.puma10_id), data=blocks[blocks.res_values > 0].groupby('puma10_id').res_values.median()).fillna(0)

    blocks['tract_vals'] = misc.reindex(tract_vals.res_values, blocks.tract_id)
    blocks['puma_vals'] = misc.reindex(puma_vals.res_values, blocks.puma10_id)

    blocks.res_values[blocks.res_values == 0] = blocks.tract_vals[blocks.res_values == 0]
    blocks.res_values[blocks.res_values == 0] = blocks.puma_vals[blocks.res_values == 0]

    return misc.reindex(blocks.res_values, residential_units.block_id)

# tract_id = pd.Series(blocks.index.values, index = blocks.index).str.slice(0,11)
# blocks['zone_id'] = tract_id.astype('int')
blocks['tract_id'] = pd.Series(blocks.index.values, index = blocks.index).str.slice(0,11)
blocks['zone_id'] = blocks.taz_id

residential_units['res_rent'] = rent(residential_units, blocks)
residential_units['res_value'] = value(residential_units, blocks)

/home/janowicz/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/janowicz/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/janowicz/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jan

In [4]:


households['zone_id'] = misc.reindex(blocks.zone_id, households.block_id)
jobs['zone_id'] = misc.reindex(blocks.zone_id, jobs.block_id)
residential_units['zone_id'] = misc.reindex(blocks.zone_id, residential_units.block_id)

del households['block_id']
del jobs['block_id']
del residential_units['block_id']

In [5]:
zone_x = blocks.groupby('zone_id').x.mean()
zone_y = blocks.groupby('zone_id').y.mean()
zone_rent = residential_units.groupby('zone_id').res_rent.mean()
zone_value = residential_units.groupby('zone_id').res_value.mean()
zone_acres = blocks.groupby('zone_id').square_meters_land.sum()/4046.86
zone_du_capacity = blocks.groupby('zone_id').residential_unit_capacity.sum()
zone_emp_capacity = blocks.groupby('zone_id').employment_capacity.sum()

blocks.puma10_id = blocks.puma10_id.astype('int')
zone_puma = blocks.groupby('zone_id').puma10_id.median()

zone_hh = households.groupby('zone_id').size()
zone_du = residential_units.groupby('zone_id').size()
zone_emp = jobs.groupby('zone_id').size()

In [6]:
puma_rent = blocks[blocks.res_rents > 0].groupby('puma10_id').res_rents.mean() * .8
puma_rent = pd.DataFrame({'res_rent':puma_rent}, index=puma_rent.index)
puma_value = blocks[blocks.res_values > 0].groupby('puma10_id').res_values.mean() * .8
puma_value = pd.DataFrame({'res_value':puma_value}, index=puma_value.index)

In [7]:
zones = pd.DataFrame(index = np.unique(blocks.zone_id))
zones['x'] = zone_x
zones['y'] = zone_y
zones['res_rent'] = zone_rent
zones['res_value'] = zone_value
zones['acres'] = zone_acres
zones['residential_unit_capacity'] = zone_du_capacity
zones['employment_capacity'] = zone_emp_capacity

zones['puma10_id'] = zone_puma

zones['hh'] = zone_hh
zones['du'] = zone_du
zones['emp'] = zone_emp

In [8]:
## Next step:  fill in nan's, check values
## Next step:  Ensure that everything is needed in the zone schema
## Next step:  Write out to zone h5

print len(zones)
zones = zones[zones.acres > 0]
print len(zones)

1957
1954


In [9]:
zones['puma_rent'] = misc.reindex(puma_rent.res_rent, zones.puma10_id)
zones['puma_value'] = misc.reindex(puma_value.res_value, zones.puma10_id)

zones.res_rent = zones.res_rent.fillna(0)
zones.res_value = zones.res_value.fillna(0)
zones.res_rent[zones.res_rent == 0] = zones.puma_rent[zones.res_rent == 0]
zones.res_value[zones.res_value == 0] = zones.puma_value[zones.res_value == 0]

/home/janowicz/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/janowicz/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
del zones['puma10_id']
del zones['hh']
del zones['du']
del zones['emp']
del zones['puma_rent']
del zones['puma_value']

In [11]:
del residential_units['res_rent']
del residential_units['res_value']

In [12]:
store2 = pd.HDFStore('zone_model_data.h5')

In [13]:
zones.index.name = 'zone_id'
store2['zones'] = zones
store2['residential_units'] = residential_units
store2['households'] = households
store2['jobs'] = jobs

In [14]:
store2['edges'] = store['edges_simple']
store2['nodes'] = store['nodes_simple']
store2['annual_household_relocation_rates'] = store['annual_household_relocation_rates']
#store2['persons'] = store['persons']
store2['places'] = store['places']
store2['region'] = store['region']

In [15]:
store2.close()